# Converting XML files to CSV 

In [1]:
import pandas as pd
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging

logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

## Convert XML files to Python Dictionaries 

In [2]:
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

def get_field(xmldict, path=None, default=None):
    if path is None: return xmldict
    try:
        for p in path:
            # Ugly namespaces are ugly, let's deal with them here
            xmldict = xmldict['{http://www.irs.gov/efile}' + p]
        return xmldict
    except KeyError:
        # If this key does not exist in the dictionary, 
        return default
        # Note: it may be faster to check ```if p not in dict``` rather
        # than set up this try/except block. I think it depends on how
        # often this fails - the more it fails, the worse the try/except does.

## Analysis

### Analyze one file

This function returns a `1 x n` DataFrame of all relevant data for a single file, which can be appended to a larger DataFrame for multi-file analysis.

In [3]:
def analyze_one(path):
    tree = et.parse(path)
    root = tree.getroot()
    #Converting from xml to dictionary
    xmldict = XmlDictConfig(root)

    
    ##############Header Information 
    #Date Information
    tax_period_begin = get_field(xmldict, ['ReturnHeader','TaxPeriodBeginDt'])
    
    #Filer Information
    filer_data = get_field(xmldict, ['ReturnHeader', 'Filer'], {})
    ein = get_field(filer_data, ['EIN'])
    business_name = get_field(filer_data, ['BusinessName', 'BusinessNameLine1Txt'])
    city = get_field(filer_data, ['USAddress', 'CityNm'])
    state = get_field(filer_data, ['USAddress', 'StateAbbreviationCd'])
    zipc = get_field(filer_data, ['USAddress', 'ZIPCd'])
    filer_list = [[tax_period_begin, ein, business_name, city, state, zipc]]
    filer_df = pd.DataFrame(filer_list, columns = ['TaxPeriodBeginDt','EIN', 'BusinessName', 'City', 'State', 'ZIPCd']) 
    
    
    ##############Return Data
    return_data = get_field(xmldict, ['ReturnData', 'IRS990'])
    if return_data is None:
        logging.warning(f'Could not find Return Data for file: {path}')
        return None
    
    #Formation Year
    formation_year = get_field(return_data, ['FormationYr'], "NA")
    
    #State of legal domicile 
    legal_dom = get_field(return_data, ['LegalDomicileStateCd'], "NA")
    
    #Related Org
    related = get_field(return_data, ['RelatedOrganizationsAmt'], 0)

    #GovernmentGrantsAmt
    gov_grants = get_field(return_data, ['GovernmentGrantsAmt'], 0)

    #FederatedCampaignsAmt
    fed_camp = get_field(return_data, ['FederatedCampaignsAmt'], 0)

    #MembershipDuesAmt
    membership = get_field(return_data, ['MembershipDuesAmt'], 0)

    #FundraisingAmt
    fundraising = get_field(return_data, ['FundraisingAmt'], 0)
    
    #NoncashContributionsAmt
    NoncashContributionsAmt = get_field(return_data, ['NoncashContributionsAmt'], 0)
    
    #AllOtherContributionsAmt
    all_other = get_field(return_data, ['AllOtherContributionsAmt'], 0)

    #TotalContributionsAmt
    total_contri = get_field(return_data, ['TotalContributionsAmt'], 0)
    
    #TotalProgramServiceRevenueAmt
    program_service_rev = get_field(return_data, ['TotalProgramServiceRevenueAmt'], 0)
    
    #CYInvestmentIncomeAmt
    CY_investment_income = get_field(return_data, ['CYInvestmentIncomeAmt'], 0)
    
    #CYOtherRevenueAmt
    CYOtherRevenueAmt = get_field(return_data, ['CYOtherRevenueAmt'], 0)
    
    #CYTotalRevenueAmt
    CYTotalRevenueAmt = get_field(return_data, ['CYTotalRevenueAmt'], 0)
    
    #CYTotalExpensesAmt
    CYTotalExpensesAmt = get_field(return_data, ['CYTotalExpensesAmt'], 0)
    
    #CYRevenuesLessExpensesAmt
    CYRevenuesLessExpensesAmt = get_field(return_data, ['CYRevenuesLessExpensesAmt'], 0)
    
    #TotalAssetsEOYAmt
    TotalAssetsEOYAmt = get_field(return_data, ['TotalAssetsEOYAmt'], 0)
    
    #TotalLiabilitiesEOYAmt
    TotalLiabilitiesEOYAmt = get_field(return_data, ['TotalLiabilitiesEOYAmt'], 0)
    
    #NetAssetsOrFundBalancesEOYAmt
    NetAssetsOrFundBalancesEOYAmt = get_field(return_data, ['NetAssetsOrFundBalancesEOYAmt'], 0)

    #MissionDesc
    descri = get_field(return_data, ['MissionDesc'], "")

    #TypeOfOrganizationCorpInd
    corp_ind = get_field(return_data, ['TypeOfOrganizationCorpInd'], "NA")
    
    #SchoolOperatingInd
    SchoolOperatingInd = get_field(return_data, ['SchoolOperatingInd'], "NA")
    
    #OperateHospitalInd
    OperateHospitalInd = get_field(return_data, ['OperateHospitalInd'], "NA") #Sometimes has {'referenceDocumentId': 'IRS990ScheduleH'} for true 
    
    #AddressChangeInd
    AddressChangeInd = get_field(return_data, ['AddressChangeInd'], "")
    
    #InitialReturnInd
    InitialReturnInd = get_field(return_data, ['InitialReturnInd'], "")
    
    #AmendedReturnInd
    AmendedReturnInd = get_field(return_data, ['AmendedReturnInd'], "")
    
    #TerminateOperationsInd
    TerminateOperationsInd = get_field(return_data, ['TerminateOperationsInd'], "")
   


    return_list = [[formation_year, legal_dom, related, gov_grants,fed_camp,membership,fundraising, NoncashContributionsAmt,all_other, total_contri,
                          program_service_rev, CY_investment_income,CYOtherRevenueAmt, CYTotalRevenueAmt,CYTotalExpensesAmt,CYRevenuesLessExpensesAmt,
                          TotalAssetsEOYAmt,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt, descri, corp_ind, SchoolOperatingInd, OperateHospitalInd,
                         AddressChangeInd, InitialReturnInd,AmendedReturnInd,TerminateOperationsInd]]
    
    return_df = pd.DataFrame(return_list, columns = ['FormationYr', 'LegalDomicileStateCd','RelatedOrganizationsAmt','GovernmentGrantsAmt',
                                                                 'FederatedCampaignsAmt','MembershipDuesAmt','FundraisingAmt','NoncashContributionsAmt',
                                                                 'AllOtherContributionsAmt','TotalContributionsAmt','TotalProgramServiceRevenueAmt','CYInvestmentIncomeAmt',
                                                                 'CYOtherRevenueAmt','CYTotalRevenueAmt','CYTotalExpensesAmt','CYRevenuesLessExpensesAmt','TotalAssetsEOYAmt',
                                                                 'TotalLiabilitiesEOYAmt','NetAssetsOrFundBalancesEOYAmt','MissionDesc','TypeOfOrganizationCorpInd', 'SchoolOperatingInd',
                                                                'OperateHospitalInd','AddressChangeInd', 'InitialReturnInd','AmendedReturnInd','TerminateOperationsInd'])

    #Concatenating header information and contributions
    return pd.concat([filer_df, return_df], axis=1, ignore_index=False)
    

### Example Analysis


In [4]:
df = pd.DataFrame([])

one_row = analyze_one('data/201601349349307825_public.xml')
df = df.append(one_row) 
df

,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,FormationYr,LegalDomicileStateCd,RelatedOrganizationsAmt,GovernmentGrantsAmt,...,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt,MissionDesc,TypeOfOrganizationCorpInd,SchoolOperatingInd,OperateHospitalInd,AddressChangeInd,InitialReturnInd,AmendedReturnInd,TerminateOperationsInd
0,2014-07-01,421264647,Covenant Medical Center Inc,Waterloo,IA,507025499,1985,IA,2036412,90353,...,26524129,230001287,"AS A MEMBER OF WHEATON FRANCISCAN HEALTHCARE, ...",X,false,{'referenceDocumentId': 'IRS990ScheduleH'},,,,false


### Get a list of files to analyze
You can generate a full index at the path used below with `extract_eins.ipynb`, or just download [this one](https://www.dropbox.com/s/237lcs9dcmqzjug/full_index.csv?dl=0) and place it in a folder called `index`. 

In [5]:
index = pd.read_csv('index/full_index.csv')
files = 'data/' + index.loc[index['TAX_YEAR'] == 2016]['OBJECT_ID'].astype('str') + '_public.xml'
files

6          data/201731359349202698_public.xml
15         data/201800529349300205_public.xml
19         data/201721359349307007_public.xml
28         data/201701869349300730_public.xml
36         data/201702209349300965_public.xml
                          ...                
3261613    data/201741309349201189_public.xml
3261621    data/201713109349302626_public.xml
3261629    data/201703189349103250_public.xml
3261637    data/201841319349304634_public.xml
3261645    data/201733199349101418_public.xml
Name: OBJECT_ID, Length: 437222, dtype: object

If you do not have all of these files available, you can do the following:

1. Run `files.to_csv('index/2017.txt', header=False, index=False)` to get a list of all the files you need to download
2. Run this in powershell: `ForEach ($file in (Get-Content -Path "index/2017.txt")) {aws s3 cp ("s3://irs-form-990/"+$file) ("data/"+$file)}` 

Note: I'm not able to completely test this script without reinstalling aws so this hasn't been completely tested but it *should* work.


### Create the dataframe with selected fields

In [6]:
df = pd.concat([one_row for name in tqdm(files) if (one_row := analyze_one(name)) is not None])
df

100%|██████████| 1000/1000 [00:04<00:00, 247.77it/s]


,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,FormationYr,LegalDomicileStateCd,RelatedOrganizationsAmt,GovernmentGrantsAmt,...,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt,MissionDesc,TypeOfOrganizationCorpInd,SchoolOperatingInd,OperateHospitalInd,AddressChangeInd,InitialReturnInd,AmendedReturnInd,TerminateOperationsInd
0,2016-06-01,010018923,THOMAS COLE POST 19,SANFORD,ME,04073,1920,ME,0,0,...,4588,318625,VETERAN'S ORGANIZATION PROVIDING SUPPORT FOR M...,X,false,false,,,,false
0,2016-01-01,010018927,AMERICAN LEGION POST 5 BOURQUE-LANIGAN,WATERVILLE,ME,04901,1926,ME,0,0,...,38161,523767,"BOURQUE-LANIGAN POST 5, AMERICAN LEGION, PROVI...",X,0,0,,,,0
0,2016-05-01,010018930,AMERICAN LEGION - MAINE DEPARTMENT,WINSLOW,ME,049010727,1919,ME,0,0,...,370923,670861,,X,false,false,,,,false
0,2016-04-01,010019705,Ancient Free and Accepted Masons of,Holden,ME,044290430,1820,ME,0,0,...,0,1743560,Provide services to all Masonic Lodges in the ...,X,0,0,,,,0
0,2016-07-01,010021545,MAINE STATE CHAMBER OF COMMERCE,AUGUSTA,ME,04330,1995,ME,0,28241,...,650527,83737,"ENHANCE THE BUSINESS CLIMATE, THE QUALITY OF L...",NA,0,0,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016-10-01,010478709,Maine Island Trail Association,Portland,ME,04103,1993,ME,0,144542,...,62892,1571685,The Maine Island Trail Association's goal is t...,X,0,0,,,,0
0,2016-07-01,010479776,MAINE EDUCATION ASSOCIATION,AUGUSTA,ME,043309412,1993,ME,0,0,...,253897,49589239,TO PROVIDE HEALTH AND OTHER BENEFITS TO ITS ME...,NA,0,0,,,,0
0,2016-01-01,010479848,RESPONSIBLE PET CARE OF OXFORD HILLS INC,SOUTH PARIS,ME,04281,NA,NA,0,57955,...,611,471030,Rescuing stray and unwanted cats and dogs. Ope...,NA,false,false,,,,false
0,2016-01-01,010481458,CHANNING CHAPEL PRESERVATION SOCIETY,WINTER HARBOR,ME,04693,1992,ME,0,0,...,0,407264,To preserve and maintain the Channing Chapel b...,NA,0,0,,,,0


In [7]:
df.to_csv("data/return_2017.csv")